In [1]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws)

Workspace.create(name='Azureml-SDK-WS02', subscription_id='f0ec0447-a406-4c0a-922d-f468c99bce13', resource_group='AzuremlSDKRG01')


In [5]:
from azureml.core import Datastore

# Create a datastore
az_store = Datastore.register_azure_blob_container(
            workspace = ws,
            datastore_name = 'azure_sdk_blob01',
            account_name = 'auremlstb01',
            container_name = 'azuremlstb01blob',
            account_key = '7JXTP7Y4dN+TLzTz2I4l1buBhLlcAOGKJoiAs+YZAWltnbpDc5PDmoE2/ODlHt1raOt8dPEi1/OS+ASt8u3e2Q==')
az_store

{
  "name": "azure_sdk_blob01",
  "container_name": "azuremlstb01blob",
  "account_name": "auremlstb01",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [14]:
ds = ws.get_default_datastore()
ds

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-c660939c-46a0-404a-80eb-a5b1b21d88cc",
  "account_name": "azuremlsstorage858c34f0b",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [18]:
ds.upload(src_dir='./loan', target_path='loan', overwrite=True, show_progress=True)


Uploading an estimated of 1 files
Uploading ./loan/Loan Approval Prediction.csv
Uploaded ./loan/Loan Approval Prediction.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_af3e68065a03401b97c50b08823a508d

In [21]:
# Create and register a dataset using SDK
from azureml.core import Dataset

# Access
# az_store = Datastore.get(ws, az_store.name)

# Create the path of the csv file
csv_path = [(ds, '/loan/Loan Approval Prediction.csv')]
print(csv_path)

# Create the dataset
loan_dataset = Dataset.Tabular.from_delimited_files(path = csv_path)

# Register the dataset
loan_dataset = loan_dataset.register(workspace = ws,
                                     name = 'Loan Applications Using SDK',
                                     create_new_version = True)

[({
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-c660939c-46a0-404a-80eb-a5b1b21d88cc",
  "account_name": "azuremlsstorage858c34f0b",
  "protocol": "https",
  "endpoint": "core.windows.net"
}, '/loan/Loan Approval Prediction.csv')]


In [22]:
loan_dataset

{
  "source": [
    "('workspaceblobstore', 'loan/Loan Approval Prediction.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "2927f7d8-0dea-4dda-82c5-7b4132a6f1ee",
    "name": "Loan Applications Using SDK",
    "version": 1,
    "workspace": "Workspace.create(name='Azureml-SDK-WS02', subscription_id='f0ec0447-a406-4c0a-922d-f468c99bce13', resource_group='AzuremlSDKRG01')"
  }
}

In [23]:
ds_list = list(ws.datasets.keys())
ds_list

['Loan Applications Using SDK']

## Access the Workspace, Datastore and Datasets

In [27]:
from azureml.core import Workspace, Datastore, Dataset, Experiment

ws = Workspace.from_config()
az_store = Datastore.get(ws, 'azure_sdk_blob01')
az_dataset = Dataset.get_by_name(ws, 'Loan Applications Using SDK')
az_default_store = ws.get_default_datastore()

## Create/Access an experiment object

In [28]:
experiment = Experiment(workspace = ws,
                       name = 'Loan-SDK-Exp01')
experiment

Name,Workspace,Report Page,Docs Page
Loan-SDK-Exp01,Azureml-SDK-WS02,Link to Azure Machine Learning studio,Link to Documentation


## Run Experiment

In [40]:
new_run = experiment.start_logging()
new_run

Experiment,Id,Type,Status,Details Page,Docs Page
Loan-SDK-Exp01,c4856770-210c-4982-ab20-c457048bfd9a,,Running,Link to Azure Machine Learning studio,Link to Documentation


In [32]:
df = az_dataset.to_pandas_dataframe()
df.head(5)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,False,0,Graduate,False,5849,0.0,NaN,360.0,1.0,Urban,True
1,LP001003,Male,True,1,Graduate,False,4583,1508.0,128.0,360.0,1.0,Rural,False
2,LP001005,Male,True,0,Graduate,True,3000,0.0,66.0,360.0,1.0,Urban,True
3,LP001006,Male,True,0,Not Graduate,False,2583,2358.0,120.0,360.0,1.0,Urban,True
4,LP001008,Male,False,0,Graduate,False,6000,0.0,141.0,360.0,1.0,Urban,True


In [33]:
# count the observations
total_observations = len(df)
total_observations

614

In [39]:
#Get the null values
nulldf = df.isnull().sum()
nulldf

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     2
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

## Log the metrics to the workspace

In [41]:
new_run.log('Total Observations', total_observations)

In [42]:
for columns in df.columns:
    new_run.log(columns, nulldf[columns])

## Complete an experiment

In [43]:
new_run.complete()